In [61]:
import pandas as pd
import numpy as np
import json
import os

%load_ext autoreload
%autoreload 1
from utils.hourly_volas_and_dt import hourly_volas_and_dt
%aimport utils.hourly_volas_and_dt

# manually entered data from CoinMarketCap and other sources, with columns 
# [Symbol, Name, Market Cap Number, Market Cap Modifier, Market Cap]
df = pd.read_csv('summaries.csv') 

df = df[df['Symbol'] != "SC"]

symbols_list_file_name = "binance_BTC_from_2019_05_01_to_2022_04_30.json"
with open(symbols_list_file_name, "r") as f:
    symbols_list = json.load(f)
symbols_list.remove('scbtc')

df.reset_index(drop=True, inplace=True)
input_folder_name = "full_returns_winsorised"

symbol_to_stats = dict()
for symbol in symbols_list:
    dv = hourly_volas_and_dt(
        symbol, input_folder_name
    ).groupby(pd.Grouper(freq="D")).sum()
    columns_to_keep = ['25%', '50%', '75%', 'mean', 'std']
    renamed = ['1Q', 'Median', '3Q', 'Mean', 'SD']
    mapping = dict(zip(columns_to_keep, renamed))
    stats = (
        dv.describe()
        .transpose()
        [columns_to_keep]
        .rename(columns=mapping)
    )
    symbol_to_stats[symbol] = stats

big_stats = pd.concat(
    symbol_to_stats
).droplevel(0).reset_index(drop=True) * 10 ** 3

to_drop = ['Market Cap Number', 'Market Cap Modifier']
df.drop(columns=to_drop, inplace=True)

df['Market Cap'] = df['Market Cap'] / 10 ** 9

to_print = pd.concat([df, big_stats], axis=1)
file_name = 'exploration/stats_table.tex'
precision_digits = 3
format_type='g'
alternate = '#'
my_formatter = f"{{:{alternate}.{precision_digits}{format_type}}}".format
to_print.to_latex(
    file_name,
    escape=False, 
    float_format=my_formatter, 
    index=False
)